In [6]:
!rm -rf 6162-KDD
!git clone https://github.com/nityanandkore/6162-KDD.git


Cloning into '6162-KDD'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 60 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [11]:
import pandas as pd
!rm abcnews-date-text.csv
!unzip '6162-KDD/DataSet/abcnews-date-text.zip'
data = pd.read_csv('abcnews-date-text.csv')

Archive:  6162-KDD/DataSet/abcnews-date-text.zip
  inflating: abcnews-date-text.csv   


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#take a look at the content of the 'headline_text' column
data_text = data[['headline_text']]

In [13]:
data_text

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
...,...
1041788,who was alphabay founder alexandre cazes
1041789,wolfe brothers berry farming country music in ...
1041790,wollongong refugee ramps up liberian charity work
1041791,women choosing diy ivf sperm inseminsation to ...


In [0]:
#add a column to data_text for the row index
data_text['index'] = data_text.index
documents = data_text

C:\Users\P2190101\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
len(documents)

1041793

In [0]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\P2190101\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [0]:
type(documents)

pandas.core.frame.DataFrame

In [0]:
#take a look at the content of a document with Index# 4310
doc_sample = documents[documents['index'] == 4310].values[0][0]

In [0]:
doc_sample

'rain helps dampen bushfires'

In [0]:
documents[documents['index'] == 4310]

,headline_text,index
4310,rain helps dampen bushfires,4310


In [0]:
#We will perform the following steps:
 #Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
 #Words that have fewer than 3 characters are removed.
 #All stopwords are removed.
 #Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
 #Words are stemmed — words are reduced to their root form.


#import relvant packages for conduct topic modeling analysis
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [0]:
from nltk.stem import*
stemmer = PorterStemmer()

In [0]:
#a function to perform lemmatize and stem preprocessing steps on the data set. You may "Google" each Python method to get the meaning of the parameters.

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [0]:
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [0]:
#remove na values from the column 'headline_text'
documents = documents.dropna(subset=['headline_text'])

In [0]:
#select a document to preview after preprocessing
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [0]:
#preprocess'headline_text', savie the results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [0]:
#create a dictionary from ‘processed_docs’ containing the number of times a word appears in the document set
dictionary = gensim.corpora.Dictionary(processed_docs)

In [0]:
dictionary

In [0]:
#Filter out tokens that appear in
 #less than 15 documents (absolute number) or
 #more than 0.5 documents (fraction of total corpus size, not absolute number).
 #after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
#For each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4005, 1)]

In [0]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4005 ("dampen") appears 1 time.


In [0]:
#run LDA using bag of words

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [0]:
#for each topic, we will explore the words occuring in that topic and its relative weight

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.049*"australia" + 0.026*"world" + 0.024*"south" + 0.022*"second" + 0.019*"news" + 0.017*"test" + 0.015*"tasmanian" + 0.014*"women" + 0.012*"leagu" + 0.011*"lead"
Topic: 1 
Words: 0.027*"market" + 0.022*"trump" + 0.021*"busi" + 0.016*"share" + 0.015*"deal" + 0.014*"support" + 0.013*"return" + 0.012*"final" + 0.012*"citi" + 0.011*"bank"
Topic: 2 
Words: 0.026*"rural" + 0.022*"hour" + 0.020*"chang" + 0.018*"fund" + 0.016*"health" + 0.015*"live" + 0.015*"say" + 0.013*"minist" + 0.013*"school" + 0.012*"govern"
Topic: 3 
Words: 0.018*"attack" + 0.017*"kill" + 0.017*"fight" + 0.013*"near" + 0.011*"premier" + 0.010*"unit" + 0.010*"donald" + 0.010*"final" + 0.009*"risk" + 0.009*"say"
Topic: 4 
Words: 0.020*"elect" + 0.018*"govern" + 0.015*"tasmania" + 0.013*"say" + 0.013*"worker" + 0.012*"labor" + 0.012*"council" + 0.011*"meet" + 0.010*"protest" + 0.009*"strike"
Topic: 5 
Words: 0.016*"farm" + 0.015*"island" + 0.013*"centr" + 0.013*"sport" + 0.012*"park" + 0.011*"win" + 0.011

In [0]:
lda_model[bow_corpus[4310]]

[(0, 0.020011147),
 (1, 0.24855973),
 (2, 0.02001444),
 (3, 0.020011147),
 (4, 0.020011682),
 (5, 0.020012455),
 (6, 0.020012716),
 (7, 0.020011147),
 (8, 0.34823185),
 (9, 0.2631237)]

In [0]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [0]:
#check the topic distribution for the Document# 4310.
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.34802770614624023	 
Topic: 0.018*"farmer" + 0.016*"water" + 0.013*"concern" + 0.013*"industri" + 0.013*"break" + 0.010*"hill" + 0.010*"resid" + 0.009*"fear" + 0.009*"royal" + 0.009*"warn"

Score: 0.2632623016834259	 
Topic: 0.036*"polic" + 0.028*"charg" + 0.025*"court" + 0.021*"death" + 0.019*"murder" + 0.017*"home" + 0.016*"countri" + 0.014*"famili" + 0.013*"accus" + 0.013*"drug"

Score: 0.24862539768218994	 
Topic: 0.027*"market" + 0.022*"trump" + 0.021*"busi" + 0.016*"share" + 0.015*"deal" + 0.014*"support" + 0.013*"return" + 0.012*"final" + 0.012*"citi" + 0.011*"bank"

Score: 0.02001442015171051	 
Topic: 0.026*"rural" + 0.022*"hour" + 0.020*"chang" + 0.018*"fund" + 0.016*"health" + 0.015*"live" + 0.015*"say" + 0.013*"minist" + 0.013*"school" + 0.012*"govern"

Score: 0.020012693479657173	 
Topic: 0.034*"australian" + 0.027*"interview" + 0.022*"crash" + 0.021*"coast" + 0.018*"die" + 0.017*"miss" + 0.015*"polic" + 0.014*"gold" + 0.014*"north" + 0.014*"fall"

Score: 0.0200124

In [0]:
#check the topic distribution for a new document (not in the corpus)

unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3503151535987854	 Topic: 0.027*"market" + 0.022*"trump" + 0.021*"busi" + 0.016*"share" + 0.015*"deal"
Score: 0.18349500000476837	 Topic: 0.049*"australia" + 0.026*"world" + 0.024*"south" + 0.022*"second" + 0.019*"news"
Score: 0.1833542287349701	 Topic: 0.034*"australian" + 0.027*"interview" + 0.022*"crash" + 0.021*"coast" + 0.018*"die"
Score: 0.18272694945335388	 Topic: 0.036*"polic" + 0.028*"charg" + 0.025*"court" + 0.021*"death" + 0.019*"murder"
Score: 0.016684820875525475	 Topic: 0.026*"rural" + 0.022*"hour" + 0.020*"chang" + 0.018*"fund" + 0.016*"health"
Score: 0.016684768721461296	 Topic: 0.018*"attack" + 0.017*"kill" + 0.017*"fight" + 0.013*"near" + 0.011*"premier"
Score: 0.016684768721461296	 Topic: 0.020*"elect" + 0.018*"govern" + 0.015*"tasmania" + 0.013*"say" + 0.013*"worker"
Score: 0.016684768721461296	 Topic: 0.016*"farm" + 0.015*"island" + 0.013*"centr" + 0.013*"sport" + 0.012*"park"
Score: 0.016684768721461296	 Topic: 0.039*"year" + 0.030*"queensland" + 0.024*"fa